In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# 1. Load và chia nhỏ văn bản
loader = TextLoader("KH001_Don_Xin_Vay_Von.txt", encoding="utf-8")
docs = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
).split_documents(loader.load())

# 2. Khởi tạo embedding model: paraphrase-multilingual-MiniLM-L12-v2
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"}  # hoặc "cuda"
)

# 3. Kết nối tới Qdrant
client = QdrantClient(host="localhost", port=6333)

# 4. Xóa và tạo lại collection với đúng dimension (384) & distance
client.recreate_collection(
    collection_name="viet_kb",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# 5. Khởi tạo Qdrant vectorstore
qdrant = Qdrant(
    client=client,
    collection_name="viet_kb",
    embeddings=embedding_model
)

# 6. Thêm documents vào vectordb
qdrant.add_documents(documents=docs, batch_size=64)

print("✅ Đã lưu embedding Vietnamese KB vào Qdrant với mô hình MiniLM!")


✅ Đã lưu embedding Vietnamese KB vào Qdrant với mô hình MiniLM!


In [5]:
import os
import glob
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from tqdm import tqdm # Thư viện để hiển thị thanh tiến trình đẹp mắt

# --- 1. Cấu hình ---
DATA_DIRECTORY = "data/data1"
COLLECTION_NAME = "viet_kb" # Đặt tên mới để không bị ghi đè collection cũ
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
CHUNK_SIZE = 1200 # Tăng kích thước chunk để chứa nhiều ngữ cảnh hơn
CHUNK_OVERLAP = 200 # Tăng độ chồng lấn để tránh cắt câu

# --- 2. Tải tất cả các file từ thư mục ---
print(f"🔍 Đang tìm và tải các file từ thư mục '{DATA_DIRECTORY}'...")
all_documents = []
# Tìm tất cả các file có đuôi .txt trong thư mục data
file_paths = glob.glob(os.path.join(DATA_DIRECTORY, "*.txt"))

if not file_paths:
    print(f"❌ Không tìm thấy file .txt nào trong thư mục '{DATA_DIRECTORY}'. Vui lòng kiểm tra lại.")
else:
    for file_path in tqdm(file_paths, desc="Đang tải các file"):
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            all_documents.extend(loader.load())
        except Exception as e:
            print(f"Lỗi khi tải file {file_path}: {e}")
    
    print(f"✅ Đã tải thành công {len(all_documents)} document từ {len(file_paths)} file.")

    # --- 3. Chia nhỏ toàn bộ văn bản ---
    print(f"🔄 Đang chia nhỏ văn bản với chunk_size={CHUNK_SIZE} và chunk_overlap={CHUNK_OVERLAP}...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", "---", ".", " "] # Thêm '---' để có thể tách các file
    )
    docs_for_db = text_splitter.split_documents(all_documents)
    print(f"✅ Đã chia thành {len(docs_for_db)} chunk để chuẩn bị embedding.")

    # --- 4. Khởi tạo embedding model ---
    print(f"🧠 Đang khởi tạo embedding model: {MODEL_NAME}...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=MODEL_NAME,
        model_kwargs={"device": "cpu"}  # Đổi thành "cuda" nếu bạn có GPU
    )
    # Lấy dimension của vector từ model. MiniLM là 384.
    embed_dim = embedding_model.client.get_sentence_embedding_dimension()
    print(f"✅ Model đã sẵn sàng. Kích thước vector: {embed_dim}")

    # --- 5. Kết nối và tạo lại collection trong Qdrant ---
    print("☁️ Đang kết nối tới Qdrant và tạo collection...")
    client = QdrantClient(host="localhost", port=6333)

    # Xóa collection cũ (nếu có) và tạo lại với cấu hình mới
    client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=embed_dim, distance=Distance.COSINE)
    )
    print(f"✅ Collection '{COLLECTION_NAME}' đã được tạo thành công.")

    # --- 6. Thêm các chunk đã xử lý vào Qdrant ---
    print("⏳ Đang thực hiện embedding và lưu vào Qdrant. Quá trình này có thể mất vài phút...")
    # LangChain Qdrant sẽ tự động xử lý việc gọi embedding_model
    qdrant = Qdrant(
        client=client,
        collection_name=COLLECTION_NAME,
        embeddings=embedding_model
    )

    qdrant.add_documents(documents=docs_for_db, batch_size=64)

    print("\n🎉 HOÀN TẤT! Toàn bộ dữ liệu đã được embedding và lưu vào Qdrant.")
    print(f"👉 Bây giờ bạn có thể sử dụng collection '{COLLECTION_NAME}' trong Langflow.")

🔍 Đang tìm và tải các file từ thư mục 'data/data1'...


Đang tải các file: 100%|██████████| 4/4 [00:00<00:00, 104.19it/s]

✅ Đã tải thành công 4 document từ 4 file.
🔄 Đang chia nhỏ văn bản với chunk_size=1200 và chunk_overlap=200...
✅ Đã chia thành 11 chunk để chuẩn bị embedding.
🧠 Đang khởi tạo embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...


✅ Model đã sẵn sàng. Kích thước vector: 384
☁️ Đang kết nối tới Qdrant và tạo collection...


/tmp/ipykernel_52034/1221667981.py:61: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Collection 'viet_kb' đã được tạo thành công.
⏳ Đang thực hiện embedding và lưu vào Qdrant. Quá trình này có thể mất vài phút...

🎉 HOÀN TẤT! Toàn bộ dữ liệu đã được embedding và lưu vào Qdrant.
👉 Bây giờ bạn có thể sử dụng collection 'viet_kb' trong Langflow.


In [4]:
import requests
import json
import time
from pprint import pprint

# --- 1. CẤU HÌNH ---

# URL API của pipeline Langflow
LANGFLOW_URL = "http://localhost:7860/api/v1/run/70017ce4-caba-479c-a03c-067faebf3c6c"

# Headers cho request, chỉ định rõ encoding UTF-8
HEADERS = {"Content-Type": "application/json; charset=utf-8"}

# Các tham số cho logic lặp
MAX_ITERATIONS = 20      # Tăng số lần lặp tối đa vì có thể phải chia nhỏ nhiều lần
INITIAL_BATCH_SIZE = 6 # Số trường thông tin hỏi trong lô ban đầu
MIN_BATCH_SIZE_TO_SPLIT = 2 # Chỉ chia nhỏ lô nếu nó còn nhiều hơn 1 trường

# Danh sách các trường thông tin cần trích xuất (hoàn toàn bằng tiếng Việt)
NEW_TEMPLATE_MAPPING = {
    "headerInfo": {
        "bbc": "BBC (Báo cáo bởi)",
        "cbc": "CBC (Cán bộ chính)",
        "idDeXuat": "ID đề xuất",
        "ngayBaoCao": "Ngày báo cáo",
        "ngayCapNhat": "Ngày cập nhật",
        "mucDichThamDinh": "Mục đích thẩm định",
        "capNoi": "Cấp nơi",
    },
    "creditInfo": {
        "idT24": "ID T24",
        "xepHangTinDung": "Xếp hạng tín dụng",
        "ngayXepHang": "Ngày xếp hạng",
        "ketQuaPhanNhomTiepCan": "Kết quả phân nhóm tiếp cận",
        "nganh": "Ngành",
        "phanNhomRuiRo": "Phân nhóm rủi ro",
        "phanNhomUngXu": "Phân nhóm ứng xử",
        "khacBietHDTD": "Khác biệt HĐTD",
        "ketQuaPhanLuong": "Kết quả phân luồng",
        "loaiKhoanVay": "Loại khoản vay",
        "tongGiaTriCapTD": "Tổng giá trị cấp TD",
        "tongGiaTriCoBPHD": "Tổng giá trị có BPHĐ",
        "xepHangRuiRo": "Xếp hạng rủi ro",
        "ruiRoNganh": "Rủi ro ngành",
        "mucDoPhucTap": "Mức độ phức tạp",
        "tieuChiTaiChinh": "Tiêu chí tài chính",
        "mucDoRuiRo": "Mức độ rủi ro",
    },
    "businessInfo": {
        "tenDayDu": "Tên đầy đủ của doanh nghiệp",
        "ngayThanhLap": "Ngày thành lập",
        "loaiHinhCongTy": "Loại hình công ty",
        "hoatDongKinhDoanhMoTa": "Mô tả hoạt động kinh doanh",
        "tienDoVanXuat": "Tiến độ vận xuất",
        "khaNangLapKeHoach": "Khả năng lập kế hoạch",
    },
    "legalInfo": {
        "tinhHinhPhapLy": "Tình hình pháp lý",
        "kinhNghiemChuSoHuu": "Kinh nghiệm chủ sở hữu",
    },
    "tcbRelationship": {
        "chatLuongQuanHeTD": "Chất lượng quan hệ TD",
        "khongViPham": "Có vi phạm không",
        "chiTietViPham": "Chi tiết vi phạm",
        "soThangTuongTacT24": "Số tháng tương tác T24",
        "soDuTienGui12Thang": "Số dư tiền gửi 12 tháng",
        "soLanPhatSinhGiaoDich": "Số lần phát sinh giao dịch",
        "tiLeCoSuDungSPDVKhac": "Tỷ lệ có sử dụng SPDV khác",
    },
    # Các trường phức tạp (array) sẽ được xử lý sau
    "management": {},
    "financialStatus": {},
}

FIELDS_TO_EXTRACT = []
# Tạo thêm reverse map: key -> section
KEY_TO_SECTION = {}
for section, fields in NEW_TEMPLATE_MAPPING.items():
    for key, value in fields.items():
        FIELDS_TO_EXTRACT.append(value)
        KEY_TO_SECTION[key] = section

# --- 2. CÁC HÀM HỖ TRỢ ---

def create_prompt(fields_list: list) -> str:
    """Tạo prompt động dựa trên số lượng trường cần hỏi."""
    if not fields_list:
        return ""
    
    # Tạo danh sách các trường dưới dạng một chuỗi text thuần túy
    fields_as_text_list = "\n- ".join(fields_list)
    
    # Thay đổi câu lệnh prompt để rõ ràng hơn với LLM
    if len(fields_list) == 1:
        # Prompt tập trung vào một trường duy nhất
        return f"{fields_as_text_list}"
    else:
        # Prompt cho một danh sách các trường
        return f"""
- {fields_as_text_list}
"""

def is_valid_value(value) -> bool:
    """Kiểm tra xem giá trị trả về có hợp lệ không (không phải None, không rỗng)."""
    return value is not None and str(value).strip() != ""

def query_langflow_for_json(question_prompt: str) -> dict:
    """
    Gửi một prompt đến Langflow, nhận phản hồi và cố gắng trích xuất một đối tượng JSON.
    (Hàm này giữ nguyên như cũ, không cần thay đổi)
    """
    if not question_prompt:
        return {}
    payload = {
        "input_value": question_prompt, "output_type": "chat", "input_type": "chat"
    }
    try:
        response = requests.post(LANGFLOW_URL, json=payload, headers=HEADERS, timeout=120)
        response.raise_for_status()
        langflow_data = json.loads(response.text)
        llm_response_text = langflow_data['outputs'][0]['outputs'][0]['results']['message']['text']
        start = llm_response_text.find('{')
        end = llm_response_text.rfind('}')
        if start != -1 and end != -1:
            json_str = llm_response_text[start : end + 1]
            return json.loads(json_str)
        else:
            print("  - Lỗi: Không tìm thấy đối tượng JSON hợp lệ trong phản hồi của LLM.")
            return {}
    except requests.exceptions.RequestException as e:
        print(f"  - Lỗi kết nối API: {e}")
        return {}
    except (KeyError, IndexError):
        print("  - Lỗi: Cấu trúc JSON trả về từ Langflow không như mong đợi.")
        pprint(langflow_data)
        return {}
    except json.JSONDecodeError as e:
        print(f"  - Lỗi: Không thể phân tích JSON từ phản hồi của LLM. {e}")
        print(f"  - Phản hồi gốc từ LLM: {llm_response_text}")
        return {}


# --- 3. LOGIC TRÍCH XUẤT "CHIA ĐỂ TRỊ" ---

def main():
    """
    Hàm chính điều khiển luồng trích xuất thông tin thông minh.
    Sử dụng một hàng đợi (queue) để quản lý các lô công việc.
    Nếu một lô thất bại, nó sẽ được chia nhỏ và thêm lại vào hàng đợi.
    """
    final_result = {}
    
    # Khởi tạo hàng đợi công việc (work queue)
    # Ban đầu, chia toàn bộ danh sách thành các lô có kích thước `INITIAL_BATCH_SIZE`
    work_queue = [
        FIELDS_TO_EXTRACT[i:i + INITIAL_BATCH_SIZE]
        for i in range(0, len(FIELDS_TO_EXTRACT), INITIAL_BATCH_SIZE)
    ]
    
    current_iteration = 0
    print("🚀 Bắt đầu quá trình trích xuất thông tin với logic 'Chia để trị'...")

    # Vòng lặp sẽ tiếp tục khi vẫn còn việc trong hàng đợi và chưa vượt quá giới hạn
    while work_queue and current_iteration < MAX_ITERATIONS:
        current_iteration += 1
        
        # Lấy lô công việc tiếp theo từ đầu hàng đợi
        current_batch = work_queue.pop(0)
        
        print(f"\n--- VÒNG LẶP {current_iteration}/{MAX_ITERATIONS} | Lô còn lại: {len(work_queue)} ---")
        print(f"Đang xử lý lô gồm {len(current_batch)} trường: {current_batch}")
        
        # Tạo prompt và gửi yêu cầu
        prompt = create_prompt(current_batch)
        response_json = query_langflow_for_json(prompt)

        newly_found_fields = []
        if response_json:
            for field in current_batch:
                # Kiểm tra xem trường có trong phản hồi và có giá trị hợp lệ không
                if field in response_json and is_valid_value(response_json[field]):
                    value = response_json[field]
                    print(f"    ✅ Đã tìm thấy '{field}': {value}")
                    final_result[field] = value
                    newly_found_fields.append(field)
        
        # *** LOGIC "CHIA ĐỂ TRỊ" NẰM Ở ĐÂY ***
        
        # Xác định các trường chưa tìm được trong lô này
        failed_fields = [f for f in current_batch if f not in newly_found_fields]
        
        if failed_fields:
            print(f"  - Không tìm thấy {len(failed_fields)} trường: {failed_fields}")
            
            # Nếu lô thất bại còn đủ lớn để chia, thì chia đôi nó ra
            if len(failed_fields) >= MIN_BATCH_SIZE_TO_SPLIT:
                print(f"  -> splitting_batch Chia nhỏ lô thất bại và thêm lại vào hàng đợi.")
                mid_point = len(failed_fields) // 2
                first_half = failed_fields[:mid_point]
                second_half = failed_fields[mid_point:]
                
                # Thêm 2 lô nhỏ hơn vào ĐẦU hàng đợi để được ưu tiên xử lý ngay
                work_queue.insert(0, second_half)
                work_queue.insert(0, first_half)
            else:
                # Nếu lô quá nhỏ để chia (chỉ còn 1 trường), ta coi như thất bại cuối cùng
                print(f"  -> Lô quá nhỏ, không chia nữa. Ghi nhận là không tìm thấy.")
                # Chúng ta sẽ báo cáo các trường này ở cuối
        
        time.sleep(3) # Tạm dừng giữa các lần gọi API

    # --- 4. TỔNG KẾT ---
    print("\n\n✅ Quá trình trích xuất hoàn tất!")
    
    # Các trường còn lại trong hàng đợi hoặc không thể tìm thấy sau khi chia nhỏ
    remaining_fields = set(FIELDS_TO_EXTRACT) - set(final_result.keys())

    if remaining_fields:
        print(f"⚠️ Không thể trích xuất {len(remaining_fields)} trường sau:")
        pprint(list(remaining_fields))

    print("-----------------------------------------")
    print("          KẾT QUẢ JSON CUỐI CÙNG         ")
    print("-----------------------------------------")
    
    print(json.dumps(final_result, indent=4, ensure_ascii=False))


if __name__ == "__main__":
    main()
    

🚀 Bắt đầu quá trình trích xuất thông tin với logic 'Chia để trị'...

--- VÒNG LẶP 1/20 | Lô còn lại: 6 ---
Đang xử lý lô gồm 6 trường: ['BBC (Báo cáo bởi)', 'CBC (Cán bộ chính)', 'ID đề xuất', 'Ngày báo cáo', 'Ngày cập nhật', 'Mục đích thẩm định']
    ✅ Đã tìm thấy 'BBC (Báo cáo bởi)': BBC-DN-HANOI
    ✅ Đã tìm thấy 'CBC (Cán bộ chính)': CBC-KHDN-LON
    ✅ Đã tìm thấy 'ID đề xuất': DX-2024-8868
    ✅ Đã tìm thấy 'Ngày báo cáo': 05 tháng 10 năm 2024
    ✅ Đã tìm thấy 'Ngày cập nhật': 02/10/2024
    ✅ Đã tìm thấy 'Mục đích thẩm định': Cấp mới

--- VÒNG LẶP 2/20 | Lô còn lại: 5 ---
Đang xử lý lô gồm 6 trường: ['Cấp nơi', 'ID T24', 'Xếp hạng tín dụng', 'Ngày xếp hạng', 'Kết quả phân nhóm tiếp cận', 'Ngành']
    ✅ Đã tìm thấy 'Cấp nơi': Hội sở chính
    ✅ Đã tìm thấy 'ID T24': 10034598
    ✅ Đã tìm thấy 'Xếp hạng tín dụng': A3
    ✅ Đã tìm thấy 'Ngày xếp hạng': 01/10/2024
    ✅ Đã tìm thấy 'Kết quả phân nhóm tiếp cận': Nhóm 2
    ✅ Đã tìm thấy 'Ngành': Rủi ro Cao

--- VÒNG LẶP 3/20 | Lô còn